In [25]:
import pandas as pd

In [26]:
# read batches weather api csv files
df = pd.concat([pd.read_csv('weatherdata/datapulls/batch1.csv'), pd.read_csv('weatherdata/datapulls/batch2.csv'), pd.read_csv('weatherdata/datapulls/batch3.csv'), pd.read_csv('weatherdata/datapulls/batch4.csv'), pd.read_csv('weatherdata/datapulls/batch5.csv')], ignore_index=True)
df.head(1)

,Unnamed: 0,date,weather,summary,icon,temperature,feels_like,wind_chill,soil_temperature,dew_point,...,wind.speed,wind.gusts,wind.angle,wind.dir,cloud_cover.total,cloud_cover.low,cloud_cover.middle,cloud_cover.high,precipitation.total,precipitation.type
0,0,2023-03-01T00:00:00,sunny,Sunny,2,-1.3,-2.7,-4.3,0.6,-2.3,...,2.2,3.8,41,NE,2,2,0,0,0.0,none


In [27]:
# localize for DST
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].dt.tz_localize("Europe/Amsterdam", ambiguous='infer').dt.tz_convert("UTC")
df['date'] = df['date'] + pd.Timedelta(hours=1)
df = df[(df['date'] >= '2023-03-21') & (df['date']  <= '2024-04-15 23:00')].reset_index(drop=True)
df

,Unnamed: 0,date,weather,summary,icon,temperature,feels_like,wind_chill,soil_temperature,dew_point,...,wind.speed,wind.gusts,wind.angle,wind.dir,cloud_cover.total,cloud_cover.low,cloud_cover.middle,cloud_cover.high,precipitation.total,precipitation.type
0,480,2023-03-21 00:00:00+00:00,overcast,Overcast,7,8.9,5.6,6.2,8.4,8.6,...,5.0,8.6,221,SW,100,100,48,79,0.0,none
1,481,2023-03-21 01:00:00+00:00,overcast,Overcast,7,8.8,5.6,6.1,8.4,8.5,...,5.0,8.5,218,SW,100,100,54,94,0.0,none
2,482,2023-03-21 02:00:00+00:00,rain,Rain,11,8.7,5.4,5.9,8.3,8.4,...,4.9,8.5,217,SW,100,100,32,100,0.6,rain
3,483,2023-03-21 03:00:00+00:00,rain,Rain,11,8.6,5.4,5.9,8.3,8.4,...,4.8,8.3,215,SW,100,100,66,100,1.0,rain
4,484,2023-03-21 04:00:00+00:00,rain,Rain,11,8.6,5.6,6.1,8.3,8.3,...,4.5,7.9,215,SW,100,100,89,96,0.6,rain
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9403,284,2024-04-15 19:00:00+00:00,cloudy,Cloudy,6,7.3,0.7,2.8,9.5,3.7,...,9.5,17.7,278,W,89,20,86,52,0.0,none
9404,285,2024-04-15 20:00:00+00:00,light_rain,Light rain,10,6.5,-0.6,1.5,8.8,4.1,...,10.2,19.0,277,W,97,46,95,86,0.3,rain
9405,286,2024-04-15 21:00:00+00:00,overcast,Overcast,7,6.3,-0.7,1.3,8.1,4.1,...,10.2,19.3,275,W,96,38,91,90,0.0,none
9406,287,2024-04-15 22:00:00+00:00,light_rain,Light rain,10,6.5,0.2,1.9,7.7,4.1,...,9.0,17.8,271,W,92,35,80,85,0.4,rain


In [28]:
# check if dataframe contains daylight saving/measurement intervals are consistent
# fine if only index 0 returned
df['time_diff'] = df['date'].diff()
dst_check = df[df['time_diff'] != pd.Timedelta(hours=1)]
df.drop(columns=['time_diff'], inplace=True)
dst_check

,Unnamed: 0,date,weather,summary,icon,temperature,feels_like,wind_chill,soil_temperature,dew_point,...,wind.gusts,wind.angle,wind.dir,cloud_cover.total,cloud_cover.low,cloud_cover.middle,cloud_cover.high,precipitation.total,precipitation.type,time_diff
0,480,2023-03-21 00:00:00+00:00,overcast,Overcast,7,8.9,5.6,6.2,8.4,8.6,...,8.6,221,SW,100,100,48,79,0.0,none,NaT


In [29]:
# mask = df[(df['date'] >= '2023-3-29 4:00') & (df['date'] <= '2023-3-30 9:00')]
# function to correct the pump data outdoor temperature readings 
# (not a necessity since those will not be used in the forecasting)
def temperature_ratio(date1, date2, folder_name):
    mask = df[(df['date'] >= date1) & (df['date'] <= date2)]
    mask = mask[['date', 'temperature']]
    mask['temp_ratio'] = mask['temperature'] / mask['temperature'].shift(1) 
    mask = mask.drop(mask.index[0])
    mask.to_csv(folder_name, index=False)

In [30]:
temperature_ratio('2023-03-29 05:00','2023-03-30 10:00','weatherdata/weatherratio.csv')
temperature_ratio('2023-04-13 11:00','2023-04-17 06:00','weatherdata/weatherratio2.csv')
temperature_ratio('2023-07-21 15:00','2023-07-23 23:00','weatherdata/weatherratio3.csv')
temperature_ratio('2023-10-04 05:00','2023-10-08 13:00','weatherdata/weatherratio4.csv')

In [31]:
### Unneccesary columns removal
# - icon: useless
# - summary: duplication of 'weather'
# - wind.dir: wind.angle contains the same data, but is more comprehensive
df.drop(columns=['Unnamed: 0','icon', 'summary', 'wind.dir'], inplace=True)

In [32]:
df['weather'].nunique()

13

In [33]:
df['weather'].unique()

array(['overcast', 'rain', 'light_rain', 'mostly_cloudy', 'cloudy',
       'partly_sunny', 'mostly_sunny', 'rain_shower', 'sunny',
       'psbl_rain', 'light_snow', 'snow', 'rain_and_snow'], dtype=object)

In [34]:
df['precipitation.type'].unique()

array(['none', 'rain', 'snow', 'rain_snow'], dtype=object)

The precipitation.type column seems to generalize the different weather types, which makes the 'precipitation.type' column abundant.

In [35]:
mostly_sunny = df[df['weather']=='mostly_sunny']['precipitation.type'].unique()
overcast = df[df['weather']=='overcast']['precipitation.type'].unique()
light_rain = df[df['weather']=='light_rain']['precipitation.type'].unique()
light_snow = df[df['weather']=='light_snow']['precipitation.type'].unique()
rain_and_snow = df[df['weather']=='rain_and_snow']['precipitation.type'].unique()

print(f"{mostly_sunny}, {overcast}, {light_rain}, {light_snow}, {rain_and_snow}")

['none'], ['none'], ['rain'], ['snow'], ['rain_snow']


In [36]:
# remove other abundant columns
df.drop(columns=['precipitation.type', 'weather'], inplace=True)
df.drop(df.index[0], inplace=True) # do if starts with 0:00:00 for future resampling
df.reset_index(drop=True, inplace=True)
df

,date,temperature,feels_like,wind_chill,soil_temperature,dew_point,surface_temperature,pressure,cape,evaporation,...,ozone,humidity,wind.speed,wind.gusts,wind.angle,cloud_cover.total,cloud_cover.low,cloud_cover.middle,cloud_cover.high,precipitation.total
0,2023-03-21 01:00:00+00:00,8.8,5.6,6.1,8.4,8.5,8.2,1015,3,0.00,...,331,97.0,5.0,8.5,218,100,100,54,94,0.0
1,2023-03-21 02:00:00+00:00,8.7,5.4,5.9,8.3,8.4,8.0,1015,3,0.00,...,329,98.0,4.9,8.5,217,100,100,32,100,0.6
2,2023-03-21 03:00:00+00:00,8.6,5.4,5.9,8.3,8.4,8.0,1014,3,0.00,...,327,98.0,4.8,8.3,215,100,100,66,100,1.0
3,2023-03-21 04:00:00+00:00,8.6,5.6,6.1,8.3,8.3,7.9,1014,3,0.00,...,326,98.0,4.5,7.9,215,100,100,89,96,0.6
4,2023-03-21 05:00:00+00:00,8.8,5.8,6.3,8.3,8.3,7.9,1013,3,0.00,...,328,96.0,4.4,8.0,204,100,100,91,100,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9402,2024-04-15 19:00:00+00:00,7.3,0.7,2.8,9.5,3.7,7.1,1000,0,0.02,...,468,77.0,9.5,17.7,278,89,20,86,52,0.0
9403,2024-04-15 20:00:00+00:00,6.5,-0.6,1.5,8.8,4.1,6.6,1001,16,0.01,...,467,84.0,10.2,19.0,277,97,46,95,86,0.3
9404,2024-04-15 21:00:00+00:00,6.3,-0.7,1.3,8.1,4.1,6.3,1001,17,0.01,...,467,85.0,10.2,19.3,275,96,38,91,90,0.0
9405,2024-04-15 22:00:00+00:00,6.5,0.2,1.9,7.7,4.1,6.5,1001,28,0.01,...,466,84.0,9.0,17.8,271,92,35,80,85,0.4


In [37]:
# resample for 6-hour periods using measures of centrality
df.set_index('date', inplace=True)
periodic_df = df.resample('6h', label='right', closed='right').agg({
    'temperature': 'mean',
    'feels_like': 'mean',
    'wind_chill' : 'mean',
    'soil_temperature' : 'mean',
    'dew_point' : 'mean',
    'surface_temperature' : 'mean',
    'surface_temperature' : 'mean',
    'pressure' : 'mean',
    'cape' : 'mean',
    'evaporation' : 'mean',
    'irradiance' : 'mean',
    'ozone' : 'mean',
    'humidity' : 'mean',
    'wind.speed' : 'mean',
    'wind.gusts' : 'mean',
    'wind.angle' : 'mean',
    'cloud_cover.total' : 'mean',
    'cloud_cover.low' : 'mean',
    'cloud_cover.middle' : 'mean',
    'cloud_cover.high' : 'mean',
    'precipitation.total' : 'sum',
}).round(1)

periodic_df = periodic_df.reset_index()
periodic_df.drop(periodic_df.index[-1], inplace=True)
periodic_df.rename(columns={
    'wind.speed' : 'wind_speed',
    'wind.gusts' : 'wind_gusts',
    'wind.angle' : 'wind_angle',
    'cloud_cover.total' : 'cloud_cover_total',
    'cloud_cover.low' : 'cloud_cover_low',
    'cloud_cover.middle' : 'cloud_cover_middle',
    'cloud_cover.high' : 'cloud_cover_high',
    'precipitation.total' : 'precipitation_total',
}, inplace=True)
periodic_df


,date,temperature,feels_like,wind_chill,soil_temperature,dew_point,surface_temperature,pressure,cape,evaporation,...,ozone,humidity,wind_speed,wind_gusts,wind_angle,cloud_cover_total,cloud_cover_low,cloud_cover_middle,cloud_cover_high,precipitation_total
0,2023-03-21 06:00:00+00:00,8.7,5.6,6.1,8.3,8.4,8.0,1014.0,3.5,0.0,...,328.2,97.0,4.7,8.2,212.5,100.0,100.0,72.0,98.2,2.9
1,2023-03-21 12:00:00+00:00,9.8,6.3,7.2,8.5,8.5,9.2,1012.3,6.2,0.1,...,334.0,90.8,5.5,10.2,201.2,100.0,70.5,92.2,99.5,2.4
2,2023-03-21 18:00:00+00:00,11.2,6.7,8.5,10.2,8.1,11.6,1010.3,2.5,0.1,...,359.2,81.0,7.1,12.5,222.3,93.0,77.3,46.7,27.7,0.0
3,2023-03-22 00:00:00+00:00,9.7,5.1,6.5,9.2,7.6,8.9,1008.7,0.0,0.0,...,342.3,86.0,7.4,12.2,219.5,84.8,28.0,12.5,69.7,0.0
4,2023-03-22 06:00:00+00:00,9.4,4.8,6.2,8.6,6.9,8.8,1005.8,6.2,0.0,...,330.2,83.7,7.2,12.0,209.2,87.7,48.2,63.3,77.2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1562,2024-04-14 18:00:00+00:00,11.8,7.9,9.9,14.3,4.2,14.0,1022.0,0.0,0.2,...,371.3,59.3,4.9,9.0,277.5,92.8,15.0,8.3,91.2,0.0
1563,2024-04-15 00:00:00+00:00,8.5,6.7,7.9,10.8,4.7,8.0,1018.0,2.7,0.0,...,401.3,76.7,1.6,3.4,242.0,87.7,7.7,56.8,80.0,0.0
1564,2024-04-15 06:00:00+00:00,7.9,4.8,5.1,9.0,5.0,7.7,1012.5,8.5,0.0,...,396.0,81.0,4.6,8.4,220.0,61.5,19.3,32.2,37.5,0.0
1565,2024-04-15 12:00:00+00:00,9.7,4.0,6.1,10.2,5.9,10.9,1005.3,36.2,0.1,...,418.8,77.0,8.7,17.6,214.2,77.5,38.3,73.0,5.7,3.4


In [38]:
periodic_df.to_csv('weatherdata/weatherdata.csv', index=False)